# Import

In [1]:
# import modules
import pandas as pd
import numpy as np
import spotipy

# jupyter notebook full-width display
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# pandas formatting
pd.set_option('display.float_format', '{:.3f}'.format)
# NOTE: underscore separaters ('_') are better than commas (',') because 
# numbers with underscores work in Python without any extra effort.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

# Data Sources

**The Billboard 100**

https://en.wikipedia.org/wiki/Billboard_Hot_100

https://www.kaggle.com/datasets/dhruvildave/billboard-the-hot-100-songs

**1.2M Songs with Metadata (csv)**

https://www.kaggle.com/datasets/rodolfofigueroa/spotify-12m-songs

**8+ M. Spotify Tracks, Genre, Audio Features (SQL)**

https://www.kaggle.com/datasets/maltegrosse/8-m-spotify-tracks-genre-audio-features


**Spotify API**

https://developer.spotify.com/documentation/web-api/

https://developer.spotify.com/console/get-search-item

https://developer.spotify.com/console/get-audio-features-track/

https://developer.spotify.com/documentation/web-api/reference/#/operations/get-audio-features

Spotipy Library:  https://spotipy.readthedocs.io/en/master/


# Data Description and Discussion

* The Billboard 100 data did not include audio features. It was combined with audio features from the following sources:
    * 1.2M Songs with Metadata (csv format)
    * 8+ M. Spotify Tracks, Genre, Audio Features (SQLite format)
    * Spotify API data gathered via the library Spotipy
* Overall, audio features was gathered for approximately 75% of songs from the Billboard 100.
    * Some songs were excluded based on data repetition issues
        * Typically this was only hard to find songs with very similar names
        * For example searching for 'Metallica The Unforgiven' and 'Metallica The Unforgiven Part 2' yielded the same Spotify id
            * It was determined that excluding these songs was less error-prone than manually fixing the issues
            * Alternatively, we could have kept 1 song. In this case, there is up to a 50% chance that the song is mislabelled, so this option appeared less favourable than dropping both repeat instances.
* A Quality Assurance (QA) check was performed on the final dataset.
    * Audio features from 100 songs were gathered from the Spotify API and compared to the datasets listed above.
    * There were 3 non-trivial issues noted in 2 of the 100 songs:
        * Madonna Live To Tell
            * A significant increase in loudness (~7 dB)
            * Appoximately 1 second different in length
            * All other audio features consistent between data sources
            * Both of these changes appear to result from remastering and re-uploading the track
                * https://artists.spotify.com/help/article/re-uploading-music
        * Lil Wayne Let It All Work Out
            * The key signature was not consistent between the 2 sources
            * The newer source (the API request from Sept 11, 2022) was correct (B major)
            * The SQL database was also different
            * My supposition is that these errors are due to the characteristics of the song:
                * atonal (most notably the sining)
                * detuned (bass pitch automation, and low-fi detuning effects)
    * Overall, there is a large degree of consistency between datasets. Furthermore, inconsistencies are all explainable with reasonable suppositions.

# Import Data
from: Kevin 820 Data Pickling.ipynb

In [2]:
# all songs with audio features (combined from 3 sources)
df_10M = pd.read_pickle('df_10M.pickle')

# all Billboard 100 lists, audio features included where possible
df_B100 = pd.read_pickle('df_B100.pickle')

# all unique songs from the Billboard 100 lists, audio features included where possible
df_B100_songs = pd.read_pickle('df_B100_songs.pickle')

# Data Description

In [3]:
# sizes of the datasets
df_10M.shape, df_B100.shape, df_B100_songs.shape

((9595992, 16), (329930, 21), (29681, 16))

In [4]:
# data types
pd.concat(
    [df_10M.dtypes, df_B100.dtypes, df_B100_songs.dtypes], 
    keys=['df_10M.dtypes', 'df_B100.dtypes', 'df_B100_songs.dtypes'],
    axis=1
)

,df_10M.dtypes,df_B100.dtypes,df_B100_songs.dtypes
id,object,object,object
song,object,object,object
artist,object,object,object
acousticness,float32,float32,float32
danceability,float32,float32,float32
duration_ms,Int32,Int32,Int32
energy,float32,float32,float32
instrumentalness,float32,float32,float32
key,Int16,Int16,Int16
liveness,float32,float32,float32


In [5]:
# Date Range for Billboard Hot 100

df_B100.date.min(), df_B100.date.max()

(Timestamp('1958-08-04 00:00:00'), Timestamp('2021-11-06 00:00:00'))

In [6]:
df_B100[['rank', 'last-week', 'peak-rank', 'weeks-on-board']].describe().loc['mean':'max'].T

,mean,std,min,25%,50%,75%,max
rank,50.502,28.866,1.000,26.000,51.000,76.000,100.000
last-week,47.593,28.055,1.000,23.000,47.000,72.000,100.000
peak-rank,40.973,29.348,1.000,13.000,38.000,65.000,100.000
weeks-on-board,9.162,7.619,1.000,4.000,7.000,13.000,90.000


In [7]:
df_10M.describe().loc['mean':'max'].T

,mean,std,min,25%,50%,75%,max
acousticness,0.421,0.374,0.000,0.034,0.336,0.817,0.996
danceability,0.528,0.190,0.000,0.396,0.545,0.676,1.000
duration_ms,97.408,159341.591,0.000,169600.000,216933.000,275080.000,19672058.000
energy,0.545,0.282,0.000,0.310,0.567,0.789,1.000
instrumentalness,0.258,0.374,0.000,0.000,0.002,0.645,1.000
key,5.237,3.542,0.000,2.000,5.000,8.000,11.000
liveness,0.210,0.180,0.000,0.096,0.129,0.262,1.000
loudness,-10.967,6.318,-60.000,-13.675,-9.196,-6.398,7.234
mode,0.661,0.473,0.000,0.000,1.000,1.000,1.000
speechiness,0.098,0.135,0.000,0.036,0.047,0.082,0.974


In [8]:
df_B100.describe().loc['mean':'max'].T

,mean,std,min,25%,50%,75%,max
rank,50.502,28.866,1.000,26.000,51.000,76.000,100.000
last-week,47.593,28.055,1.000,23.000,47.000,72.000,100.000
peak-rank,40.973,29.348,1.000,13.000,38.000,65.000,100.000
weeks-on-board,9.162,7.619,1.000,4.000,7.000,13.000,90.000
acousticness,0.278,0.275,0.000,0.041,0.178,0.468,0.995
danceability,0.603,0.149,0.000,0.507,0.611,0.708,0.988
duration_ms,6410.969,66552.146,30213.000,183360.000,221306.000,258399.000,1561133.000
energy,0.625,0.199,0.007,0.479,0.643,0.786,0.999
instrumentalness,0.034,0.139,0.000,0.000,0.000,0.001,0.985
key,5.218,3.564,0.000,2.000,5.000,8.000,11.000


In [9]:
df_B100_songs.describe().loc['mean':'max'].T

,mean,std,min,25%,50%,75%,max
acousticness,0.316,0.290,0.000,0.052,0.224,0.556,0.995
danceability,0.590,0.152,0.000,0.491,0.598,0.697,0.988
duration_ms,24075.435,68403.261,30213.000,169533.000,210426.000,251333.000,1561133.000
energy,0.611,0.203,0.007,0.463,0.624,0.775,0.999
instrumentalness,0.039,0.151,0.000,0.000,0.000,0.001,0.985
key,5.196,3.556,0.000,2.000,5.000,8.000,11.000
liveness,0.197,0.168,0.012,0.091,0.132,0.255,0.999
loudness,-8.927,3.622,-30.346,-11.314,-8.554,-6.111,2.291
mode,0.741,0.438,0.000,0.000,1.000,1.000,1.000
speechiness,0.067,0.076,0.000,0.032,0.040,0.061,0.951


**Proportion of Songs With Audio Feature Data:**

~75% of songs on the Billboard list are available on Spotify, and weren't removed for data errors

In [10]:
# All Billboard 100 lists
# number not null, total, proportion not null
(
    df_B100[df_B100.id.notnull()].shape[0], 
    df_B100.shape[0], 
    df_B100[df_B100.id.notnull()].shape[0] / df_B100.shape[0]
)

(253254, 329930, 0.7675991877064832)

In [11]:
# All songs from Billboard 100 lists
# number not null, total, proportion not null
(
    df_B100_songs[df_B100_songs.id.notnull()].shape[0], 
    df_B100_songs.shape[0], 
    df_B100_songs[df_B100_songs.id.notnull()].shape[0] / df_B100_songs.shape[0]
)

(22189, 29681, 0.7475826286176341)

# Exploratory Data Analysis